In [268]:
import pandas as pd
import numpy as np
import os
import time
import re

from scipy.sparse import load_npz

In [269]:
user_movie_matrix = pd.read_csv("collbarfilter.csv")
user_similarity = pd.read_csv("UtU_similarityMatrix.csv")
movies = pd.read_csv(os.path.join("data","movies.csv"))
ratings = pd.read_csv(os.path.join("data","ratings.csv"))

In [270]:
userids = ratings["userId"].unique()
user_similarity.columns = userids
user_similarity.index = userids
user_similarity

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
1,0.000000,0.000000,0.0,0.932810,0.970770,0.939243,0.951037,0.978123,0.000000,0.000000,...,0.992413,0.958830,0.927490,0.954033,0.937549,0.967379,0.958247,0.956573,0.000000,0.976881
2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.981042,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.990501
3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.932810,0.000000,0.0,0.000000,0.901137,0.899828,0.945977,0.000000,0.000000,0.000000,...,0.879535,0.918902,0.912397,0.000000,0.952565,0.933792,0.913567,0.867322,0.000000,0.921235
5,0.970770,0.000000,0.0,0.901137,0.000000,0.932712,0.000000,0.943665,0.000000,0.000000,...,0.000000,0.939417,0.933872,0.954653,0.930918,0.960953,0.954314,0.928047,0.979824,0.960010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.967379,0.000000,0.0,0.933792,0.960953,0.947985,0.931303,0.967077,0.977616,0.902706,...,0.989826,0.946998,0.951495,0.980066,0.966624,0.000000,0.952440,0.950909,0.985496,0.975492
607,0.958247,0.000000,0.0,0.913567,0.954314,0.961672,0.972062,0.963498,0.000000,0.000000,...,0.994381,0.955249,0.923803,0.958146,0.932317,0.952440,0.000000,0.952135,0.979129,0.959225
608,0.956573,0.000000,0.0,0.867322,0.928047,0.911192,0.922770,0.959250,0.939601,0.852762,...,0.979492,0.946875,0.915620,0.926622,0.929948,0.950909,0.952135,0.000000,0.972303,0.957767
609,0.000000,0.000000,0.0,0.000000,0.979824,0.965591,0.000000,0.967007,0.000000,0.000000,...,0.000000,0.968857,0.945054,0.987983,0.000000,0.985496,0.979129,0.972303,0.000000,0.963104


In [271]:
def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]","", title)

In [272]:
def personRating(user1, user2):
    return user_similarity.loc[user1, user2]

def movieSimilarity(movie_id, user_id):    
    usersSeen= ratings.loc[ratings["movieId"] == movie_id, ["userId", "rating"]]
    
    if usersSeen.shape[0] < 50:
        print("Not enough reviews")
        return
    
    if usersSeen["userId"].isin([user_id]).any():
        print(f"User {user_id} has already seen movie {movie_id}")
        return
    
    similarityScores = []
    
    for user in usersSeen["userId"]:
        similarityScores.append(personRating(user_id, user))
    
    total_similarity = np.sum(similarityScores)
    
    predicted_score = np.dot(usersSeen["rating"], similarityScores)/total_similarity
    
    return predicted_score

In [273]:
class recommendation():
    def __init__(self, SimilarityMatrix, ratings, movies) -> None:
        self.SimilarityMatrix = SimilarityMatrix
        self.ratings = ratings
        self.user = None
        self.movie_id = None
        self.movie_title = None
        self.relevantUsers = None
    
    def run(self, user_id):
        self.user = user_id
        movie_recs = {}
        
        unseen_movies = set(self.ratings["userId"].unique()) - set(self.ratings[ratings["userId"] == self.user]["movieId"])
        for movie in unseen_movies:
            score = movieSimilarity(movie, self.user)
            if score is None or score < 4:
                continue
            movie_recs[movie] = score
        
        return movie_recs

In [274]:
recs = recommendation(SimilarityMatrix= user_similarity, ratings = ratings, movies = movies)
unseen_movies = recs.run(user_id=1)

Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough reviews
Not enough r

In [277]:
movie_recs

{111: 4.1172721973314435,
 293: 4.016493619380413,
 318: 4.440454663174378,
 541: 4.084966330211355}

In [278]:
movies[movies["movieId"]==318]

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama


In [282]:
movies[movies["movieId"]==111]

,movieId,title,genres
98,111,Taxi Driver (1976),Crime|Drama|Thriller


In [283]:
movies[movies["movieId"]==4.08]

,movieId,title,genres


In [284]:
user1 = ratings[ratings["userId"] ==1]

user1.merge(movies, how = "left", on = "movieId")

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
227,1,3744,4.0,964980694,Shaft (2000),Action|Crime|Thriller
228,1,3793,5.0,964981855,X-Men (2000),Action|Adventure|Sci-Fi
229,1,3809,4.0,964981220,What About Bob? (1991),Comedy
230,1,4006,4.0,964982903,Transformers: The Movie (1986),Adventure|Animation|Children|Sci-Fi
